# Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
    
import time
import copy
import numpy as np

import matplotlib.pyplot as plt
from cycler import cycler
color_list  = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']
marker_list = ['o','x','*','v','^','d']
plt.rc('axes', prop_cycle=(cycler('color',color_list)))

from LRT import LRT
from LRT import moments
from LRT import figfuns

**Setup parameter struct:**

In [ ]:
par = LRT.ParStruct()
SAMPLE = '_p100'

## Load data

In [ ]:
# a. load
data = LRT.dataStruct()
data.logY = np.transpose(np.genfromtxt(f'data/logY_unbalanced{SAMPLE}.csv',delimiter=','))

par.T, par.N = data.logY.shape
print(f'(T,N) = ({par.T},{par.N})')

# b. rescale
Y = np.exp(data.logY)
Y_lev = Y
Y = Y / np.nanmean(Y[0,:])
data.logY = np.log(Y)
    
# c. update par
par.simN = max([par.N,par.simN])
par.k = 5
par.k_lead = 5
par.depth = 10

# Estimate

In [ ]:
model = LRT.estimate(par,data,name='LRT',color=color_list[0])
model.marker = marker_list[0]
model.name_short = 'LRT'

# Simulate

In [ ]:
model.data = LRT.simulate(model.par,model, data)

## Moments

In [ ]:
moments.calc_all(par, data,printprogress=True)
moments.calc_all(model.par,model.data,printprogress=True)

## Figures 

In [ ]:
prefix = 'unbalanced'
models = [model]

figfuns.age_profile(par, data, models, 0, 
                    varname_str='logY', prefix=prefix)

figfuns.age_profile(par, data, models, 1, 
                    varname_str='dlogY', prefix=prefix)

figfuns.age_profile(par, data, models, 5, 
                    varname_str='dlogY', prefix=prefix)

figfuns.heterogenous(par, data, models,
                     'dlogY', 0, 'life-time growth (demeaned)',
                     prefix=prefix, bounds=(-1.5,1.5))

figfuns.heterogenous(par, data, models,
                     'autocorr', 1, 'first-order autocorr.',
                     prefix=prefix, bounds=(-1,1))

figfuns.heterogenous(par, data, models,
                     'autocorr_level', 1, 'first-order autocorr. (levels)',
                     prefix=prefix, bounds=(-.25,1))

figfuns.heterogenous(par, data, models,
                     'std_dlogY', 1, 'std. of income growth',
                     prefix=prefix, bounds=(0,0.5))